In [1]:
from typing import Optional, List, Dict, Generator, Any, Tuple
from collections import defaultdict, deque
import pathlib

import requests
from pydantic import HttpUrl

from api_connection import NotionAPIConnector
from api_result_fetcher import ResultFetcher
from dict_deque import DictQueueStructure
from updater import DictDequeueStructureUpdater

connector = NotionAPIConnector()
raw_response:dict = connector.main()
headers = connector.headers

fetcher = ResultFetcher(raw_response=raw_response, headers=headers)

res = fetcher.main()

In [2]:
updater = DictDequeueStructureUpdater(response_result_fetcher=res)

t = updater.update_and_save()
t

defaultdict(collections.deque,
            {'Ton per': deque([{'page ton père': 'https://www.notion.so/page-ton-p-re-50da195ac4d54448a0e255cbc9e65bb5'}]),
             'ta mere': deque([{'ta mere': 'https://www.notion.so/ta-mere-80497c9312324710b14db45a27563932'}]),
             'Page test': deque([{'Page test': 'https://www.notion.so/Page-test-e54dd41c3ac247bbb7218cbb744990de'},
                    {'Salut les peds': 'https://www.notion.so/Salut-les-peds-a04171d69b3040dfbbd90e7577bcbfd3'},
                    {'Propal Raphael ': 'https://www.notion.so/Propal-Raphael-e266eee4b5134c48a7937dfa05b5240f'},
                    {'ma bite t': 'https://www.notion.so/ma-bite-t-7c547acf50de431a8b9559858a152330'}])})

In [10]:
import utils

class InterfaceUser:
    
    """ 
    Bridge between the user and the data structure
    Take the up to date data structure, modify it as the user interact with it 
    """

    BASE_MENU = "{}) All subjects\n{}) Shuffle the order for the current scope\n{}) Delete a subject from this session\n{}) Back to the default scope\n"

    
    PATH_DICT_COUNT = pathlib.Path.cwd() / "data" / "count_dict_structure.pkl"


    
    def __init__(self, updated_data_structure:defaultdict[str, deque]):
        self.updated_data_structure = updated_data_structure
    
    
    def main(self):
        

        count_dict:dict[str, int] = InterfaceUser.update_or_create_count_dict(updated_data_structure=self.updated_data_structure, path_pickle_dict=InterfaceUser.PATH_DICT_COUNT)
        menu = InterfaceUser.create_menu(updated_data_structure=self.updated_data_structure, count_dict=count_dict)
        return menu 
    
    @staticmethod
    def update_or_create_count_dict(updated_data_structure:defaultdict[str, deque],
                                     path_pickle_dict:pathlib.Path) -> dict[str, int]:
        """ 
        To keep track of the number of session that the user haven't read about a given subject
        """
        if not pathlib.Path.exists(path_pickle_dict):
            
            dict_to_save = {subject : 0 for subject in updated_data_structure}
            
            utils.save_pickle(object_=dict_to_save, path_to_save=InterfaceUser.PATH_DICT_COUNT)
            
            return dict_to_save
        
        else:
            saved_dict:dict = utils.load_pickle(path_object=InterfaceUser.PATH_DICT_COUNT)
            
            subject:str
            for subject in updated_data_structure:
                saved_dict.setdefault(subject, 0)
                
                
            return saved_dict
            

                
            
    @staticmethod
    def create_menu(updated_data_structure:defaultdict[str, deque],
                    count_dict:dict[str, int]) -> str:
        
        number_of_elements:int = len(InterfaceUser.BASE_MENU.split("\n")) + len(updated_data_structure)
        
        subjects_to_add_to_menu:str = "\n".join(["{}) " + subject + f" (last seen ; {count_dict[subject]} session(s) ago)"  for subject in updated_data_structure])
        
        final_menu:str = InterfaceUser.BASE_MENU + subjects_to_add_to_menu
        return final_menu.format(*list(range(number_of_elements)))

In [11]:
machin = InterfaceUser(updated_data_structure=t)
m = machin.main()
print(m)

0) All subjects
1) Shuffle the order for the current scope
2) Delete a subject from this session
3) Back to the default scope
4) Ton per (last seen ; 0 session(s) ago)
5) ta mere (last seen ; 0 session(s) ago)
6) Page test (last seen ; 0 session(s) ago)
